In [1]:
import pandas as pd #dataframes

In [2]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/yellow_tripdata_2020-"+month+".csv" , low_memory = False), ignore_index=True)

#rename the location id to match the taxi zone location id    
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [3]:
#create a new df from selected columns
df2 = df[['LocationID', 'tpep_pickup_datetime', 'passenger_count']]

#change the datetime to date
df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date
yellow_taxi_2020=df2[['tpep_pickup_datetime', 'passenger_count']]
yellow_taxi_2020.head()

<ipython-input-3-98aa50a968b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date


,tpep_pickup_datetime,passenger_count
0,2020-03-01,1.0
1,2020-03-01,1.0
2,2020-03-01,1.0
3,2020-03-01,2.0
4,2020-03-01,0.0


In [4]:
#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
yellow_taxi_2020_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)

#drop unnecessary column
yellow_taxi_2020_boroughs = yellow_taxi_2020_boroughs.drop(['LocationID'], axis=1)
yellow_taxi_2020_boroughs.head()

,tpep_pickup_datetime,passenger_count,Borough
0,2020-03-01,1.0,Manhattan
1,2020-03-01,2.0,Manhattan
2,2020-03-01,2.0,Manhattan
3,2020-03-01,2.0,Manhattan
4,2020-03-01,2.0,Manhattan


In [5]:
#create a dataframe with only usage
yellow_2020= pd.DataFrame(df2.tpep_pickup_datetime.value_counts().reset_index().values, columns=["Date", "Yellow_taxi_2020"])
yellow_2020 = yellow_2020.sort_values(by="Date")
yellow_2020

,Date,Yellow_taxi_2020
135,2008-12-31,6
126,2009-01-01,19
136,2020-01-27,5
129,2020-01-28,18
133,2020-01-29,10
...,...,...
143,2020-08-08,1
145,2020-08-27,1
137,2020-09-20,3
146,2020-10-11,1


In [6]:
#save the dataframe as csv file
yellow_2020.to_csv("yellow_2020.csv")

In [7]:
#create a dataframe to get the usage by borough
yellow_2020_borough = yellow_taxi_2020_boroughs.groupby("Borough").count().reset_index()
yellow_2020_borough.rename(columns={'tpep_pickup_datetime':'Yellow_Taxi_2020'}, inplace=True)
yellow_2020_borough=yellow_2020_borough.drop(['passenger_count'], axis=1)
yellow_2020_borough

,Borough,Yellow_Taxi_2020
0,Bronx,36596
1,Brooklyn,84249
2,EWR,386
3,Manhattan,3742788
4,Queens,246686
5,Staten Island,1193
6,Unknown,31518


In [8]:
#save the dataframe as csv file
yellow_2020_borough.to_csv("yellow_2020_borough.csv")